# Import basic libraries

Let's import some basic stuff

## Fix for `fiona`

*`fiona` has an issue with GDAL 3.0. Better set your `GDAL_DATA` path to fiona's installation (contains GDAL 2.4.4) prior to running this script.*

In [1]:
import geopandas
import osmnx as ox

# Load the OSM data

This will load the osm file to a network

In [117]:
roads_graph = ox.graph_from_file('../data/Breda/Breda extract.osm', simplify=False)
edges = ox.graph_to_gdfs(roads, nodes=False, node_geometry=False)

edges = edges.to_crs("EPSG:28992")

Investigate

In [119]:
edges.head()

,u,v,key,osmid,name,highway,area,oneway,length,geometry,maxspeed,landuse,service,bridge,access,tunnel,lanes,ref,width
0,689995792,689990659,0,126559191,Havermarkt,pedestrian,yes,False,5.549,"LINESTRING (112471.505 400203.536, 112466.591 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,689995792,689990659,1,267783561,NaN,NaN,NaN,False,5.549,"LINESTRING (112471.505 400203.536, 112466.591 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,689995792,2731861671,0,126559191,Havermarkt,pedestrian,yes,False,5.977,"LINESTRING (112471.505 400203.536, 112477.359 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,689995792,2731861671,1,267778236,NaN,NaN,NaN,False,5.977,"LINESTRING (112471.505 400203.536, 112477.359 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,689995792,2731859065,0,267778236,NaN,NaN,NaN,False,14.574,"LINESTRING (112471.505 400203.536, 112472.478 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Load the BGT data

In [125]:
bgt_roads = geopandas.read_file('../data/Breda/bgt_roads.geojson')

len(bgt_roads)

38859

# Intersect roads network with polygons

In [121]:
roads = geopandas.overlay(edges, bgt_roads, how='intersection')

 Examine the resulting dataset:

In [131]:
roads_dissolved = roads.dissolve(by='gml_id')

Export the intersected roads to GeoJSON

In [139]:
# roads_dissolved.to_file('output/roads_dissolved.geojson', driver='GeoJSON')
roads_dissolved['gml_id']

KeyError: 'gml_id'

# Export to CityJSON

Let's export everything to CityJSON.

First, we'll define how the established intersected lines will be translated to `Road` objects:

In [99]:
def get_id(feature):
    if isinstance(feature['osmid'], list):
        return str(feature['osmid'][0])
    else:
        return str(feature['osmid'])

def process_line(line, vertices):
    points = [[x, y, 0] for x, y in list(line.coords)]
    indices = [i + len(vertices) for i in range(len(points))]
    for p in points:
        vertices.append(p)
    
    return indices

def create_cityobject(feature, vertices):
    if feature['geometry'].type == "LineString":
        indices = [ process_line(feature['geometry'], vertices) ]
    else:
        indices = []
        for line in feature['geometry'].geoms:
            indices.append(process_line(line, vertices))

    return {
        "type": "Road",
        "attributes": {
            "osm_id": feature['osmid']
        },
        "geometry": [
            {
                "type": "MultiLineString",
                "lod": "0.1",
                "boundaries": indices
            }
        ]
    }

Now, let's run this against all intersected road segments:

In [134]:
vertices = []

objects = {f['gml_id']: create_cityobject(f, vertices) for i, f in roads_dissolved.iterrows()}

KeyError: 'gml_id'

Finally, let's export everything as CityJSON:

In [124]:
import json
import io

output = {
  "type": "CityJSON",
  "version": "1.0",
  "CityObjects": objects,
  "vertices": vertices
}

with open('output/breda.json', 'w') as file:
    json.dump(output, file)